In [55]:
import stageemi
import stageemi.dev.distance_wwmf as distance_wwmf
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd
from datetime import datetime
import glob 
import time

In [2]:
path="../WWMF/"
time_str='202001260000'
fname=sorted(glob.glob(path+"*"+'20200126000000'+"*"))[0]

## On ouvre les fichiers 

In [52]:
zone_dpt="41"
fname_mask='../GeoData/zones_sympo_multiples/'+zone_dpt+'_mask_zones_sympos.nc'
da_mask=xr.open_dataarray(fname_mask,chunks={"id":1})

da_mask['latitude']=da_mask['latitude'].round(5)
da_mask['latitude']=da_mask['latitude'].round(5)

In [53]:
ds=xr.open_dataset(fname,chunks={"step":1})
ds['latitude']=ds['latitude'].round(5)
ds['longitude']=ds['longitude'].round(5)

In [20]:
ds_masked=(ds*da_mask.sel(id="departement").drop("id"))
ds_distance = distance_wwmf.get_pixel_distance_dept(ds_masked,"compas")

In [21]:
ds_distance_chunk = ds_distance.chunk({"step":1})


In [22]:
distance_by_step_id = (ds_distance_chunk * da_mask).sum(['latitude',"longitude"]).compute()

/home/vincent-chabot/Documents/environnement_conda/env/geoview/lib/python3.8/site-packages/dask/array/core.py:3901: PerformanceWarning: Increasing number of chunks by factor of 31
  result = blockwise(


## On affiche la distance minimale pour chacune des timestep/region

In [23]:
distance_by_step_id.min("wwmf_2")

<xarray.DataArray (step: 48, id: 31)>
array([[ 2510.,   600.,    60., ...,  2376.,  2473.,  1911.],
       [ 2129.,   112.,    60., ...,  1961.,  2093.,  2018.],
       [ 4642.,  1454.,   472., ...,  4480.,  4618.,  3208.],
       ...,
       [ 9489.,  1964.,   730., ...,  8925.,  8699.,  7428.],
       [11590.,  2304.,  1296., ..., 10708., 11404.,  8976.],
       [10647.,  1302.,   756., ..., 10030.,  9855.,  9355.]])
Coordinates:
  * step               (step) timedelta64[ns] 01:00:00 ... 2 days 00:00:00
    surface            int64 0
    heightAboveGround  int64 2
    time               datetime64[ns] 2019-10-01
    valid_time         datetime64[ns] 2019-10-01T12:00:00
  * id                 (id) object 'departement' ... '4102+4103+4104+4105+4106'

## On affiche le temps sensible correspondant 

In [39]:
distance_by_step_id.wwmf_2[distance_by_step_id.argmin("wwmf_2")]

<xarray.DataArray 'wwmf_2' (step: 48, id: 31)>
array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [6, 8, 6, ..., 6, 6, 6],
       [6, 3, 6, ..., 6, 6, 6],
       [6, 6, 3, ..., 6, 6, 3]])
Coordinates:
    surface            int64 0
    heightAboveGround  int64 2
    wwmf_2             (step, id) int64 4 4 4 4 4 4 4 4 4 ... 6 3 3 6 6 6 6 6 3
    time               datetime64[ns] 2019-10-01
    valid_time         datetime64[ns] 2019-10-01T12:00:00
  * step               (step) timedelta64[ns] 01:00:00 ... 2 days 00:00:00
  * id                 (id) object 'departement' ... '4102+4103+4104+4105+4106'

# Checking on Loir et Cher (FRB05)

In [25]:
da_mask_ref = xr.open_dataset("../GeoData/nc_departement/FRB05.nc")["mask"].squeeze("id")
da_mask_ref['latitude']=da_mask_ref['latitude'].round(5)
da_mask_ref['longitude']=da_mask_ref['longitude'].round(5)
ds=xr.open_dataset(fname)
ds['latitude']=ds['latitude'].round(5)
ds['longitude']=ds['longitude'].round(5)
dtemp=ds*da_mask_ref
ds_distance_ref = distance_wwmf.get_pixel_distance_dept(dtemp,"compas")

In [26]:
distance_by_step_wwmf = ds_distance_ref.sum(["longitude","latitude"])

In [30]:
distance_by_step_wwmf.min("wwmf_2").compute()

<xarray.DataArray (step: 48)>
array([ 2228.,  2051.,  4513.,  9363., 12744.,  8687.,  8040.,  8982.,
       10310., 10682., 12637.,  6097.,  3860.,  2599.,  3106.,    42.,
           0.,     0.,   164.,  1666.,  5380.,  4428.,  1540.,  1718.,
        1318.,   858.,  1906.,  4654.,  5644.,  7388.,  3565.,  4293.,
        5720.,  8102.,  5954.,  3766.,   400.,  2294.,  7054.,  3160.,
        1264.,  1942.,   334.,  3236.,  8816.,  9340., 11394., 10409.])
Coordinates:
  * step               (step) timedelta64[ns] 01:00:00 ... 2 days 00:00:00
    surface            int64 0
    heightAboveGround  int64 2
    id                 <U5 'FRB05'

In [43]:
distance_by_step_wwmf["wwmf_2"][distance_by_step_wwmf.argmin("wwmf_2")]

<xarray.DataArray 'wwmf_2' (step: 48)>
array([4, 4, 4, 4, 3, 3, 3, 6, 6, 6, 6, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 3, 3, 3, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6])
Coordinates:
    surface            int64 ...
    heightAboveGround  int64 ...
    id                 <U5 'FRB05'
    wwmf_2             (step) int64 4 4 4 4 3 3 3 6 6 6 ... 8 6 6 6 6 6 6 6 6 6
  * step               (step) timedelta64[ns] 01:00:00 ... 2 days 00:00:00

## On charge le fichier du Loir et Cher
A priori on a bien la même chose 

In [35]:
df_LoirCher = pd.read_csv("../notebook_gabriel/scores_advanced/FRB05_scores_202001260100.csv")

In [51]:
df_LoirCher[["WME","WME_dist"]]

,WME,WME_dist
0,4,2228
1,4,2051
2,4,4513
3,4,9363
4,3,12744
5,3,8687
6,3,8040
7,6,8982
8,6,10310
9,6,10682


In [19]:
def to_categorical(y, num_classes=None, dtype='float32'):
  """Converts a class vector (integers) to binary class matrix.
  E.g. for use with categorical_crossentropy.
  Arguments:
      y: class vector to be converted into a matrix
          (integers from 0 to num_classes).
      num_classes: total number of classes.
      dtype: The data type expected by the input. Default: `'float32'`.
  Returns:
      A binary matrix representation of the input. The classes axis is placed
      last.
  """
  y = np.array(y, dtype='int')
  input_shape = y.shape
  if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
    input_shape = tuple(input_shape[:-1])
  y = y.ravel()
  if not num_classes:
    num_classes = np.max(y) + 1
  n = y.shape[0]
  categorical = np.zeros((n, num_classes), dtype=dtype)
  categorical[np.arange(n), y] = 1
  output_shape = input_shape + (num_classes,)
  categorical = np.reshape(categorical, output_shape)
  return categorical

def conversion(ds,name):
    """convert wwmf into wme (compas) or w1 (agat) code"""
    
    file_CodesWWMF= '../utils/CodesWWMF.csv'
    df_WWMF = pd.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
    if name=="compas":
        var_name="wme_arr"
        col_name="Code WME"
        ds[var_name]=ds.unknown
    elif name=="agat":
        var_name="w1_arr"
        col_name="Code W1"
        ds[var_name]=ds.unknown       
        
    elif name=="compas_asym":
        var_name="wme_asym_arr"
        col_name="Code WME"
        ds[var_name]=ds.unknown      
    elif name=="agat_asym":
        var_name="w1_asym_arr"
        col_name="Code W1"
        ds[var_name]=ds.unknown 

    for iwwmf,wwmf in enumerate(df_WWMF["Code WWMF"]):
        ds[var_name]=ds[var_name].where(ds.unknown!=wwmf,df_WWMF[col_name][iwwmf])
    return ds

def xarray_cat(da,mask=None):
    y = da.astype(np.uint8).values
    cat = to_categorical(y)
    sh = cat.shape 
    dout = xr.Dataset()
    dout[da.name] = (tuple(list(da.dims) + ["wwmf"]),cat)
    for dim in da.dims: 
        dout[dim] = da[dim]
    dout["wwmf"] = range(sh[-1])
    #dout = dout.sel(wwmf=slice(1,sh[-1]))# On vire l'indice 0
    return dout 
    
def get_matrix(name):
    """
    Retourne la matrice de distance sous forme de xarray 
    """
    if name == "compas":
        fname_dist = '../utils/distance_compas.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')
        var_name="wme_arr"
        varsh="wme_c_"
        
    elif name == "agat":
        fname_dist = '../utils/distance_agat.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')  
        var_name="w1_arr"
        varsh="w1_c_"
        
    elif name == "compas_asym":
        fname_dist = '../utils/distance_compas_asym.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')
        var_name="wme_asym_arr"
        varsh="wme_asym_c_"
        
    elif name == "agat_asym":
        fname_dist = '../utils/distance_agat_asym.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')  
        var_name="w1_asym_arr"
        varsh="w1_asym_c_"

    col_name = df_dist.columns[0]
    index = df_dist[col_name]
    values = df_dist.set_index(col_name).values
    mat = xr.Dataset()
    mat["distance"] = (("wwmf","wwmf_2"),values)
    mat["wwmf"] = index.values
    mat["wwmf_2"] = index.values
    mat["distance"].attrs["var_name"] = var_name
    return mat["distance"]

def get_pixel_distance_dept(ds_dept,name):
    " On donne en entree le dataset du departement"
    dconvert = conversion(ds_dept,name)
    distance_matrix = get_matrix(name)
    dcat = xarray_cat(dconvert[distance_matrix.var_name])
    return dcat[distance_matrix.var_name].dot(distance_matrix)